In [0]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import pickle
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

img_size=512

In [0]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    data_format='channels_last',
    #brightness_range=[0.5, 1.5],
    validation_split =0.2
)


In [0]:
train_generator = train_datagen.flow_from_directory(
'/content/covidx',
target_size=(img_size,img_size),
batch_size=16,
color_mode = 'rgb',
shuffle = True,
seed = 42,
class_mode='binary',
subset = 'training')


valid_generator = train_datagen.flow_from_directory(
directory="/content/covidx",
target_size=(img_size, img_size),
color_mode="rgb",
batch_size=16,
class_mode="binary",
shuffle=True,
seed=42,
subset ='validation'

)


In [0]:
test_datagen = ImageDataGenerator(
    #horizontal_flip=True,
    #zoom_range=[0.5, 1.25],
    rescale=1./255, 
    #preprocessing_function=func
 )
test_generator = test_datagen.flow_from_directory(
    directory="/content/drive/My Drive/train",
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=1,
    class_mode='binary',
    shuffle=False,
    seed=42
    )


In [0]:
import tensorflow
from tensorflow.python.keras.models import Sequential,Model
from tensorflow.python.keras.layers import Conv3D,Conv2D,MaxPooling2D,Input,Dense,Flatten,Dropout,Activation,BatchNormalization,GlobalAveragePooling2D
from tensorflow.python.keras.layers import PReLU
from tensorflow.python.keras import losses
from tensorflow.keras.optimizers import Adam, Adagrad
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import keras
from tensorflow.python.keras.layers import LeakyReLU, PReLU
import os
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.callbacks import TensorBoard


base_model1=VGG16(include_top=False, input_shape=(img_size,img_size,3),classes=1000,weights='imagenet')
base_model1.summary()


In [0]:

#  only training some layers 

for layers in base_model1.layers[:-7]:
  layers.trainable=False
  
for layers in base_model1.layers:
  print(layers ,layers.trainable)
  
  

In [0]:
model = Sequential(layers=base_model1.layers)
model.summary()

In [0]:
def createmodel():
  #     #  here we added new layers
  model.add(GlobalAveragePooling2D())
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid', name ='output'))
  
  opt = Adam(lr=1e-6,decay= 1e-6/100)
  model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
  #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
  model.summary()
  return model


In [0]:
model=createmodel()

## **Training** **process** **starts** **here**

In [0]:
model.fit_generator(
        train_generator,
        steps_per_epoch=139,
        epochs=15,
        validation_data=valid_generator,
        validation_steps=35
        )

In [0]:
model.save('xraymodel.hdf5') #saving model

In [0]:
model = tensorflow.keras.models.load_model("/content/drive/My Drive/xraymodel.hdf5") #to load the  saved model

In [0]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

model = tf.keras.models.load_model('/content/drive/My Drive/b.hdf5')

prediction=model.predict(test_generator,steps=110,verbose=1)
print(prediction)


In [0]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

# **Confusion** **matrix**

In [0]:
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes)
  plt.yticks(tick_marks, classes)

  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")
  else:
    print('Confusion matrix, without normalization')
    
  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="red" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()
y_pred = (prediction>0.5)*1
cm=confusion_matrix(test_generator.classes,y_pred)
cm_labels= ['COVID19','OTHERS']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix')
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix', normalize=True)

# **Performance** **of** **model**

In [0]:
print('Classification Report')
target_names = ['COVID19','OTHERS']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))